In [1]:
import csv
import os
import pandas as pd
import re
import sys
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
def process_json(dirname, fname):
    temp_df = pd.read_json(os.path.join(dirname, fname), lines = True).drop(["url"], axis = 1).dropna().replace('\n', ' ', regex=True)

#     # Prepare label in fastText format
#     temp_df["party"] = ['__label__'+s for s in temp_df["party"]]

    return temp_df

# Setup sql
engine = create_engine('sqlite:///wikidb.db')

session = sessionmaker()
session.configure(bind=engine)
s = session()

In [9]:
dirnames = ["data_extract/text/AA", "data_extract/text/AB"]

for dirname in dirnames:
    for fname in os.listdir(dirname):
        temp_df = process_json(dirname, fname)

        temp_df.to_sql("wikidb", con = engine, index = False, if_exists = "append", chunksize = 1000)

    #     temp_df.to_csv(f'{os.path.basename(fname)}.txt', index=False, sep=' ', 
    #                    header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

        del temp_df

    s.commit()

In [3]:
# Test that the db exists

pd.read_sql_query(f'SELECT COUNT(*) FROM wikidb', engine)

,COUNT(*)
0,143411


In [5]:
pd.read_sql_query(f'SELECT * FROM wikidb LIMIT 10', engine)

,id,text,title
0,303777,Virus classification Virus classification is ...,Virus classification
1,303783,Enterovirus Enteroviruses are a group of viru...,Enterovirus
2,303786,"Explode (song) ""Explode"" is a song recorded b...",Explode (song)
3,303789,Kingdom of Italy (1861–1946) The Kingdom of I...,Kingdom of Italy (1861–1946)
4,303889,New Jersey Turnpike The New Jersey Turnpike i...,New Jersey Turnpike
5,303919,Selectivity Selectivity may refer to:,Selectivity
6,303932,Jordan at the Olympics Jordan at the Olympics...,Jordan at the Olympics
7,303936,Kazakhstan at the Olympics Kazakhstan was fir...,Kazakhstan at the Olympics
8,303938,North Korea at the Olympics North Korea at th...,North Korea at the Olympics
9,303940,South Korea at the Olympics South Korea at th...,South Korea at the Olympics
